In [ ]:
#Uploading the data.
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/'
datafile = url + 'UCI%20HAR%20Dataset.zip'
!curl $datafile --output UCI_HAR_Dataset.zip
!unzip -qq UCI_HAR_Dataset.zip
!mv -f UCIn HARn Dataset UCI_HAR_DATASETY

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('UCI HAR Dataset/test/y_test.txt')-1
ytrain = np.loadtxt('UCI HAR Dataset/train/y_train.txt')-1
# load the x,y,z body accelerations test data
xx=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt')
yy=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_y_test.txt')
zz=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_z_test.txt')
# concatenate the arrays along the last dimension
xtest = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)
# follow the same approach for the train data
xx=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt')
yy=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt')
zz=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt')
xtrain = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)

In [ ]:
print(ytest.shape)
print(ytrain.shape)
print(xtest.shape)
print(xtrain.shape)

In [ ]:
from sklearn.model_selection import  train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.2)

In [ ]:
print(xvalid.shape)
print(yvalid.shape)

In [ ]:
ytrain_in = tf.keras.utils.to_categorical(ytrain)
yvalid_in = tf.keras.utils.to_categorical(yvalid)

xtrain_in = (np.reshape(xtrain, (xtrain.shape[0], 384))).astype(np.float32)
xvalid_in = (np.reshape(xvalid, (xvalid.shape[0], 384))).astype(np.float32)

In [ ]:
print(xtrain_in.shape)
print(ytrain_in.shape)
print(xvalid_in.shape)
print(yvalid_in.shape)

In [ ]:
print(ytrain_in)
print(yvalid_in)

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
print(W)
print(W.shape)
print(b.shape)

In [ ]:
def y_pred (x):
  return tf.nn.softmax(tf.matmul(x, W) + b)

def loss(x,y):
    y_ = y_pred(x)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))

In [ ]:
logdir = 'tflogs'
writer = tf.summary.create_file_writer(logdir)

In [ ]:
train_steps = 10000
lr = 1e-3
optimizer = tf.optimizers.SGD(lr)


In [ ]:
def accuracy (x, y):
  y_ = y_pred(x)
  correct = tf.math.equal(tf.math.argmax(y_, axis = -1), tf.math.argmax(y, axis = -1))
  correct = tf.cast(correct, dtype = tf.float32)
  return tf.math.reduce_mean(correct)

In [ ]:
with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)        
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)  
    if i%100 == 0:
      print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
#Activation of Tensorboard and plot of the loss function.
%load_ext tensorboard
%tensorboard --logdir tflogs/

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
#Trying Adam optimizer
optimizer = tf.optimizers.Adam(lr)

with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)     
    if i%100 == 0:
       print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
#Trying 5000 train steps
train_steps = 5000
lr = 1e-3
optimizer = tf.optimizers.Adam(lr)

with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)     
    if i%100 == 0:
       print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
#Trying 15000 train steps
train_steps = 15000
lr = 1e-3
optimizer = tf.optimizers.Adam(lr)

with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)     
    if i%100 == 0:
       print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
#Trying Learning rate 0.1
train_steps = 5000
lr = 1e-1
optimizer = tf.optimizers.Adam(lr)

with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)     
    if i%100 == 0:
       print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
#Trying Learning rate 0.01
train_steps = 5000
lr = 1e-2
optimizer = tf.optimizers.Adam(lr)

with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)     
    if i%100 == 0:
       print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
#Trying Learning rate 0.0001
train_steps = 5000
lr = 1e-4
optimizer = tf.optimizers.Adam(lr)

with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)     
    if i%100 == 0:
       print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
W = tf.Variable(tf.zeros([384, 6], name = 'W'))
b = tf.Variable(tf.zeros([6]), name = 'b')

In [ ]:
#chosen model
train_steps = 5000
lr = 1e-3
optimizer = tf.optimizers.Adam(lr)

with writer.as_default():
  for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain_in, ytrain_in)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    tf.summary.scalar('current_loss', current_loss, step = i)
    acc = accuracy(xtrain_in, ytrain_in)
    tf.summary.scalar('acc', acc, step = i)      
    if i%100 == 0:
       print('Training Step:' + str(i) + '  Loss = ' + str(current_loss) + '  Accuracy = ' + str(acc))

In [ ]:
print(accuracy(xtrain_in, ytrain_in))
print(accuracy(xvalid_in, yvalid_in))

In [ ]:
#Initial model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 2), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 10, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Tuning epochs = 100, patience = 4
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 4), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Tuning epochs = 100, patience = 6
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 6), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Tuning epochs = 100, patience = 8
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 8), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Tuning epochs = 100, patience = 10
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Tuning epochs = 100, patience = 15
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Tuning epochs = 100, patience = 20
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 20), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience 15, tuning batch size = 16
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 16, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience 15, tuning batch size = 64
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 64, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, tuning batch size = 128
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 128, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, tuning learning rate = 0.1
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.1), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, tuning learning rate = 0.01
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, tuning learning rate = 0.001
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, tuning learning rate = 0.0001
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4, activation = tf.nn.relu))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod12 = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod12 == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, learning rate = 0.0001. Two more layers. Double filters and Kernel size.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu))
model.add(tf.keras.layers.MaxPool1D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, learning rate = 0.0001. Two more layers. Double filters and Kernel size.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu))
model.add(tf.keras.layers.MaxPool1D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, learning rate = 0.0001. Two more layers. Double filters and Kernel size.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu))
model.add(tf.keras.layers.MaxPool1D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(xtrain, ytrain_in, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid, yvalid_in))

predictions = model.predict(xvalid)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod == yvalid)/len(yvalid))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
ytest = np.loadtxt('UCI HAR Dataset/test/y_test.txt')-1
ytrain = np.loadtxt('UCI HAR Dataset/train/y_train.txt')-1

xx=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt')
yy=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_y_test.txt')
zz=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_acc_z_test.txt')
xxt=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/total_acc_x_test.txt')
yyt=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/total_acc_y_test.txt')
zzt=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/total_acc_z_test.txt')
xxg=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_gyro_x_test.txt')
yyg=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_gyro_y_test.txt')
zzg=np.loadtxt('/content/UCI HAR Dataset/test/Inertial Signals/body_gyro_z_test.txt')

xtest2 = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None],
xxt[:,:,None],yyt[:,:,None],zzt[:,:,None],
xxg[:,:,None],yyg[:,:,None],zzg[:,:,None]),axis=2)

In [ ]:
xx=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt')
yy=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt')
zz=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt')
xxt=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt')
yyt=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt')
zzt=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt')
xxg=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt')
yyg=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt')
zzg=np.loadtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_gyro_z_train.txt')

xtrain2 = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None],
xxt[:,:,None],yyt[:,:,None],zzt[:,:,None],
xxg[:,:,None],yyg[:,:,None],zzg[:,:,None]),axis=2)


In [ ]:
xtrain2, xvalid2, ytrain2, yvalid2 = train_test_split(xtrain2, ytrain, test_size = 0.2)

In [ ]:
ytrain_in2 = tf.keras.utils.to_categorical(ytrain2)
yvalid_in2 = tf.keras.utils.to_categorical(yvalid2)

In [ ]:
print(ytest.shape)
print(xtest2.shape)
print(ytrain.shape)
print(xtrain2.shape)
print(yvalid2.shape)
print(xvalid2.shape)

In [ ]:
#Epochs = 100 with patience = 15, batch size = 32, learning rate = 0.0001. Two more layers. Double filters and Kernel size.
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu))
model2.add(tf.keras.layers.MaxPool1D())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 8), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model2.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model2.fit(xtrain2, ytrain_in2, epochs = 100, batch_size = 32, callbacks = callbacks, verbose = 0, validation_data = (xvalid2, yvalid_in2))

predictions = model2.predict(xvalid2)
y_pred_mod2 = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod2 == yvalid2)/len(yvalid2))

In [ ]:
#Epochs = 100 with patience = 8, batch size = 16, learning rate = 0.0001. Two more layers. Double filters and Kernel size.
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu))
model2.add(tf.keras.layers.MaxPool1D())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 8), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model2.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model2.fit(xtrain2, ytrain_in2, epochs = 100, batch_size = 16, callbacks = callbacks, verbose = 0, validation_data = (xvalid2, yvalid_in2))

predictions = model2.predict(xvalid2)
y_pred_mod2 = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod2 == yvalid2)/len(yvalid2))

In [ ]:
#Epochs = 100 with patience = 8, batch size = 64, learning rate = 0.0001. Two more layers. Double filters and Kernel size.
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu))
model2.add(tf.keras.layers.MaxPool1D())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 8), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model2.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model2.fit(xtrain2, ytrain_in2, epochs = 100, batch_size = 64, callbacks = callbacks, verbose = 0, validation_data = (xvalid2, yvalid_in2))

predictions = model2.predict(xvalid2)
y_pred_mod2 = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod2 == yvalid2)/len(yvalid2))

In [ ]:
#Epochs = 100 with patience = 8, batch size = 128, learning rate = 0.0001. Two more layers. Double filters and Kernel size.
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 8, activation = tf.nn.relu, padding = 'same', input_shape = (128, 9)))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 8, activation = tf.nn.relu))
model2.add(tf.keras.layers.MaxPool1D())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 8), tf.keras.callbacks.TensorBoard(log_dir=logdir)]

model2.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model2.fit(xtrain2, ytrain_in2, epochs = 100, batch_size = 128, callbacks = callbacks, verbose = 0, validation_data = (xvalid2, yvalid_in2))

predictions = model2.predict(xvalid2)
y_pred_mod2 = np.argmax(predictions, axis = -1)
print('Validation Accuracy: ', np.sum(y_pred_mod2 == yvalid2)/len(yvalid2))

In [ ]:
%tensorboard --logdir tflogs/

In [ ]:
ytest_in = tf.keras.utils.to_categorical(ytest)
xtest_in = (np.reshape(xtest, (xtest.shape[0], 384))).astype(np.float32)

print('Test Accuracy:', accuracy(xtest_in, ytest_in))

In [ ]:
predictions = model.predict(xtest)
y_pred_mod = np.argmax(predictions, axis = -1)
print('Test Accuracy: ', np.sum(y_pred_mod == ytest)/len(ytest))

In [ ]:
predictions = model2.predict(xtest2)
y_pred_mod2 = np.argmax(predictions, axis = -1)
print('Test Accuracy: ', np.sum(y_pred_mod2 == ytest)/len(ytest))